In [7]:
from sklearn import cross_validation
from sklearn import linear_model
import time
import csv

In [8]:
execfile('ML_Challenge_data_preprocessing.py')

('Generated X with shape ', (347698, 1000))
('Generated y with shape ', (347698,))


In [9]:
bigrams = []
with open('possible_bigrams.csv', 'rb') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        bigrams = row

bigrams.sort()
bigrams_dict = {key: i for i, key in enumerate(bigrams)}
bigrams_dict

{'AA': 0,
 'AC': 1,
 'AG': 2,
 'AT': 3,
 'CA': 4,
 'CC': 5,
 'CG': 6,
 'CT': 7,
 'GA': 8,
 'GC': 9,
 'GG': 10,
 'GT': 11,
 'TA': 12,
 'TC': 13,
 'TG': 14,
 'TT': 15}

In [24]:
X2 = []
y2 = []
for i, x in enumerate(X):
    if 'N' not in X[i,:]:
        X2.append(x)
        y2.append(y[i])
        
X2 = np.array(X2)
y2 = np.array(y2)
print X2.shape
print y2.shape

(347683, 1000)
(347683,)


In [25]:
execfile('ngram_features.py')
# for every observation, create a new feature vector or length len(bigrams_dict.keys())

sample_size = 10000#X.shape[0])

random_sample_idcs = np.random.choice(X2.shape[0], sample_size)
X_sample = np.zeros((sample_size, len(bigrams_dict)))
y_sample = np.zeros((sample_size))

start = time.clock()

for i in range(sample_size):
    rand_idx = random_sample_idcs[i]    
    X_sample[i,:] = ngram_features(X2[rand_idx,:], 2, bigrams_dict)
    y_sample[i] = y2[rand_idx]

finish = time.clock()
print 'time spent creating new features for {0} observations: {1}'.format(sample_size, finish - start)
# will take about 11 minutes for the whole data set
print X_sample.shape
print y_sample.shape

time spent creating new features for 10000 observations: 20.164328
(10000, 16)
(10000,)


In [29]:
test_size = 0.1
y_sample = y[0:sample_size]
X_train, X_test, y_train, y_test = cross_validation.train_test_split(
    X_sample, y_sample, test_size=test_size, random_state=0)
print X_train.shape
print y_train.shape

(9000, 16)
(9000,)


# Logistic Regression

In [69]:
import time

start_time = time.clock()
logreg = linear_model.LogisticRegression()
print logreg.fit(X_train, y_train)
end_time = time.clock()

print 'time spent to train ' + str(1 - test_size) + ': ' + str(end_time - start_time) + ' seconds.'

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0)
time spent to train 0.9: 0.117471 seconds.


In [70]:
logreg.score(X_test, y_test)

0.84599999999999997

# SUPPORT VECTOR MACHINE - RBF

In [32]:
from sklearn.svm import SVC

start_time = time.clock()
clf = SVC(kernel = 'rbf')
print clf.fit(X_train, y_train)
end_time = time.clock()

print 'time spent to train ' + str(1 - test_size) + ': ' + str(end_time - start_time) + ' seconds.'

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0, degree=3, gamma=0.0,
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)
time spent to train 0.9: 9.883949 seconds.


In [33]:
clf.score(X_test, y_test)

0.84099999999999997

# SUPPORT VECTOR MACHINE - Linear

In [34]:
from sklearn.svm import SVC

start_time = time.clock()
clf = SVC(kernel = 'linear')
print clf.fit(X_train, y_train)
end_time = time.clock()

print 'time spent to train ' + str(1 - test_size) + ': ' + str(end_time - start_time) + ' seconds.'

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0, degree=3, gamma=0.0,
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)
time spent to train 0.9: 131.252274 seconds.


In [35]:
clf.score(X_test, y_test)

0.84599999999999997

# Decision Tree Classifier

In [36]:
from sklearn.tree import DecisionTreeClassifier

start_time = time.clock()
clf = DecisionTreeClassifier(max_depth=5)
print clf.fit(X_train, y_train)
end_time = time.clock()

print 'time spent to train ' + str(1 - test_size) + ': ' + str(end_time - start_time) + ' seconds.'

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            random_state=None, splitter='best')
time spent to train 0.9: 0.0478 seconds.


In [37]:
clf.score(X_test, y_test)

0.84499999999999997

# ADABOOST

In [59]:
from sklearn.ensemble import AdaBoostClassifier

start_time = time.clock()
clf = AdaBoostClassifier()
print clf.fit(X_train, y_train)
end_time = time.clock()

print 'time spent to train ' + str(1 - test_size) + ': ' + str(end_time - start_time) + ' seconds.'

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=None)
time spent to train 0.9: 0.40559 seconds.


In [60]:
clf.score(X_test, y_test)

0.84499999999999997

# RANDOM FOREST CLASSIFIER

In [81]:
from sklearn.ensemble import RandomForestClassifier

start_time = time.clock()
clf = RandomForestClassifier(n_estimators=100, criterion='gini', max_features=4)
print clf.fit(X_train, y_train)
end_time = time.clock()

print 'time spent to train ' + str(1 - test_size) + ': ' + str(end_time - start_time) + ' seconds.'

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=4, max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
time spent to train 0.9: 2.330437 seconds.


In [82]:
clf.score(X_test, y_test)

0.84099999999999997

# GAUSSIAN NAIVE BAYES

In [42]:
from sklearn.naive_bayes import GaussianNB

start_time = time.clock()
clf = GaussianNB()
print clf.fit(X_train, y_train)
end_time = time.clock()

print 'time spent to train ' + str(1 - test_size) + ': ' + str(end_time - start_time) + ' seconds.'

GaussianNB()
time spent to train 0.9: 0.011296 seconds.


In [43]:
clf.score(X_test, y_test)

0.84099999999999997

In [72]:
from sklearn import neighbors

max_neighbors=22
inc_by = 3
num_neighbors = range(3,max_neighbors,inc_by)
accuracies = []
for k in num_neighbors:
    clf = neighbors.KNeighborsClassifier(k)
    clf.fit(X_train, y_train)
    start_fit = time.clock()
    score = clf.score(X_test, y_test)
    finish_fit = time.clock()
    accuracies.append(score)    
    print 'Accuracy for ' + str(k) + ' neighbors: ' + str(score)
    print 'Time spent calculating neighbors: ' + str(finish_fit - start_fit)
    print ''

Accuracy for 3 neighbors: 0.812
Time spent calculating neighbors: 0.108339

Accuracy for 6 neighbors: 0.843
Time spent calculating neighbors: 0.08607

Accuracy for 9 neighbors: 0.841
Time spent calculating neighbors: 0.11688

Accuracy for 12 neighbors: 0.845
Time spent calculating neighbors: 0.110182

Accuracy for 15 neighbors: 0.846
Time spent calculating neighbors: 0.12159

Accuracy for 18 neighbors: 0.846
Time spent calculating neighbors: 0.126474

Accuracy for 21 neighbors: 0.846
Time spent calculating neighbors: 0.123477



# GRADIENT BOOSTING MACHINE

In [89]:
from sklearn.ensemble import GradientBoostingClassifier 

start_time = time.clock()
clf = GradientBoostingClassifier(n_estimators=10, learning_rate=1.0, max_depth=2)
print clf.fit(X_train, y_train)
end_time = time.clock()

print 'time spent to train ' + str(1 - test_size) + ': ' + str(end_time - start_time) + ' seconds.'

GradientBoostingClassifier(init=None, learning_rate=1.0, loss='deviance',
              max_depth=2, max_features=None, max_leaf_nodes=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=10,
              random_state=0, subsample=1.0, verbose=0, warm_start=False)
time spent to train 0.9: 0.116106 seconds.


In [90]:
clf.score(X_test, y_test)

0.84199999999999997